<a href="https://colab.research.google.com/github/OverflowCat/parseDicts/blob/slave/tibet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ebooklib
import ebooklib
from ebooklib import epub
import binascii

     |████████████████████████████████| 112kB 5.9MB/s 
  Created wheel for ebooklib: filename=EbookLib-0.17.1-cp36-none-any.whl size=38163 sha256=19a11aaa4267abf0c09044215c76e848ae799493c3205b0fcb315b25ba772dce
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
Successfully built ebooklib


In [2]:
!mkdir writemdict

In [3]:
!ls

drive  sample_data  writemdict


In [4]:
!cd writemdict
!git clone https://github.com/zhansliu/writemdict --depth=1
!cd ..

Cloning into 'writemdict'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
Unpacking objects: 100% (12/12), done.
remote: Total 12 (delta 0), reused 7 (delta 0), pack-reused 0


In [5]:
import sys
sys.path.insert(1, 'writemdict')

In [6]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup

In [7]:
book = epub.read_epub('./drive/MyDrive/mdict/tibet/Gangjong.epub')

In [8]:
docs = book.get_items_of_type(ebooklib.ITEM_DOCUMENT)

In [9]:
def convert_x(text):
  text = text.replace('\\x', '')  # str == 'e6898ee58aa0e68b89'
  text = text.encode('utf8')  # str == b'e6898ee58aa0e68b89'
  text = binascii.a2b_hex(text)  # str == b'\xe6\x89\x8e\xe5\x8a\xa0\xe6\x8b\x89'
  text = text.decode('utf8')  # str == '扎加拉'
  return text

In [57]:
xmls = []
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
      # print('==================================')
      # print('NAME : ', item.get_name())
      xmls.append(item.get_content().decode("utf-8"))
dictxml = xmls[1:-1]

In [58]:
dictxml[0]

'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><div>\n\t\t\t<div class="frame-2">\n\t\t\t\t<a href="http://www.tibetanebook.com">\n\t\t\t\t\t<img class="frame-1" src="image/Tibetan-ebook-Logo-Smal_fmt.png" alt="Tibetan-ebook-Logo-Small_template.jpg"/>\n\t\t\t\t</a>\n\t\t\t</div>\n\t\t\t<p class="Body"/>\n\t\t\t<p class="Body"/>\n\t\t\t<p id="toc_marker-1" class="Chapter">རྩོམ་སྒྲིག་པའི་གསལ་བཤད།</p>\n\t\t\t<p class="Body"/>\n\t\t\t<p class="Body">  </p>\n\t\t\t<p class="Body">    ཚིག་མཛོད་ནི་དུས་རབས་རིམ་བྱུང་གི་ལོ་རྒྱུས་ཐོག་ཏུ་བེད་སྤྱོད་བྱས་དང་བྱེད་བཞིན་པའི་ཐ་སྙད་ཀྱི་བང་མཛོད་ཅིག་རེད།  འཇིག་རྟེན་འདིར་སྐད་ཡིག་ཡོད་པའི་མི་རིགས་ཤིག་ཡིན་ཕྱིན་རང་གི་སྐད་ཡིག་ཏུ་ཡོད་པའི་ཐ་སྙད་རྣམས་ཕྱོགས་སྒྲིག་དང་འགྲེལ་བཤད་བྱས་ནས་ཕྱི་རབས་རྗེས་འཛིན་པ་རྣམས་ཀྱི་ནོར་སྐལ་དུ་འཇོག་ཅིང་ཡོད་པ་ནི་འགྲོ་བ་མིའི་རིག

In [27]:
cons = [0, 1, 2, 3, 4, 5, 6, 7]
pros = cons[1:-2]
pros

[1, 2, 3, 4, 5]

In [78]:
def deleteFront(string, front):
  front_len = len(front)
  if string[:front_len] == front:
    res = string[front_len:]
  return res.strip()

In [83]:
# DEBUG

entries = []
xml = dictxml[256]
soup = BeautifulSoup(xml)
ps = soup.select('p.Body')
print(len(ps))
for p in ps:
  span = p.span
  if span:
    span_str = span.string
    span.extract()
    entry = (span_str, p.string.strip())
  else:
    # 单词的结尾是 ་，།，་།， 
    text = p.string
    print(p)
    tsheg_position = text.find("་")
    shad_position = text.find("།")
    # print(tsheg_position, shad_position)
    if False:
      if shad_position == -1:
        if tsheg_position == -1:
          cut_position = text.find(" ")
        else:
          cut_position = tsheg_position
      else:
        pos_delta = abs(shad_position - tsheg_position)
        if pos_delta  <= 1:
          cut_position = max(tsheg_position, shad_position)
        else:
          cut_position = min(tsheg_position, shad_position)
    else:
      cut_position = shad_position
    print(cut_position)
    entry = (text[:cut_position + 1], text[cut_position + 1:])
  print(entry)
  entries.append(entry)

118
<p class="Body">ཅུ།  རྗེས་འཇུག་ཡོད་པའི་གྲངས་འཛིན་མིང་མཐར་བཅུ་ཞེས་པའི་ཚབ་ཏུ་འགྲོ་བ།    སུམ་ཅུ།  དྲུག་ཅུ།  བདུན་ཅུ།    བརྒྱད་ཅུ། </p>
2
('ཅུ།', '  རྗེས་འཇུག་ཡོད་པའི་གྲངས་འཛིན་མིང་མཐར་བཅུ་ཞེས་པའི་ཚབ་ཏུ་འགྲོ་བ།    སུམ་ཅུ།  དྲུག་ཅུ།  བདུན་ཅུ།    བརྒྱད་ཅུ། ')
<p class="Body">ཅུ་གང་།  སྨྱུག་བཅུད་དེ་རྩི་སྨན་གྱི་རིགས་ཤིག   རོ་མངར་ལ་བསྐ།  ཞུ་རྗེས་བསིལ་ལ་སྙོམས།  ནུས་པས་གློ་ཚད་དང་།    རྨ་ཚད།  ཚད་པ་རུས་ལ་ཞེན་པ།  རིམས་ནད་རྨོང་བུ་བཅས་ལ་ཕན།  མིང་གི་རྣམ་གྲངས་ལ་སྨྱིག་མ་དང་།  རྩི་འཇམ།  བཾ་ཤ་རོ་ཙ་ན་བཅས་སོ།། </p>
6
('ཅུ་གང་།', '  སྨྱུག་བཅུད་དེ་རྩི་སྨན་གྱི་རིགས་ཤིག   རོ་མངར་ལ་བསྐ།  ཞུ་རྗེས་བསིལ་ལ་སྙོམས།  ནུས་པས་གློ་ཚད་དང་།    རྨ་ཚད།  ཚད་པ་རུས་ལ་ཞེན་པ།  རིམས་ནད་རྨོང་བུ་བཅས་ལ་ཕན།  མིང་གི་རྣམ་གྲངས་ལ་སྨྱིག་མ་དང་།  རྩི་འཇམ།  བཾ་ཤ་རོ་ཙ་ན་བཅས་སོ།། ')
<p class="Body">ཅུ་གང་གི་གཟུགས་བརྙན།  རྒྱུ་ཅུ་གང་ལས་གྲུབ་པའི་མི་སྣ་དང་།    མི་སྣའི་ལུས་ཀྱི་ཆ་ཤས་ཁག་བཟོས་པའི་སྒྱུ་རྩལ་གྱི་སྣང་བརྙན་ལ་ཟེར།    དེ་ནི་དཀར་པོ་དང་ཚོན་མདོག་ཡོད་པའི་རིགས་གཉིས་དབྱེ་ཡོད།    ཅུ་གང་གི་གཟུགས་བརྙན་དཀར་པོ་ནི་རྟག་ཏུ་མཛེས་རྩལ་སློབ་མ་ཚོས་ནག་རིས་སྦྱོ

In [99]:
entries = []
counter = 0
for xml in dictxml:
  soup = BeautifulSoup(xml)
  ps = soup.select('p.Body')
  for p in ps:
    if str(p) == '<p class="Body"></p>':
      continue
    span = p.span
    if span is not None:
      span_str = span.string
      span.extract()
      entry = (span_str, p.text.strip())
    else:
      # 单词的结尾是 ་，།，་།， 
      text = p.string
      if text is None:
        text=p.text
        print(p)
        continue
      tsheg_position = text.find("་")
      shad_position = text.find("།")
      # print(tsheg_position, shad_position)
      if False:
        if shad_position == -1:
          if tsheg_position == -1:
            cut_position = text.find(" ")
          else:
            cut_position = tsheg_position
        else:
          pos_delta = abs(shad_position - tsheg_position)
          if pos_delta  <= 1:
            cut_position = max(tsheg_position, shad_position)
          else:
            cut_position = min(tsheg_position, shad_position)
      else:
        cut_position = shad_position
      entry = (text[:cut_position + 1], text[cut_position + 1:])
    entries.append(entry)
    counter = counter + 1
counter

79893

In [28]:
# DERECATED
entries = []
blankinfo = []
for xml in dictxml:
  soup = BeautifulSoup(xml)
  ps = soup.select('p.Body')
  for p in ps:
    entry = {}
    span = p.span
    if span is not None:
      n = span.text
      if len(n) > 1:
        if n[-1] == "།":
          n = n[0:-1]
        if n[-1] == "་":
          n = n[0:-1]
      entry["a"] = n
    else:
      entry["a"] = ""
    h = p.decode_contents()
    entry["b"] = h.split("</span>")[-1].strip()
    if entry["a"] == "" and entry["b"] == "":
      continue
    elif entry["a"] == "":
      blankinfo.append(p)
    elif entry["a"] == "":
      print (f"条目 {entry['a']} 解释为空")
    # if True or (entry["a"] != "" and entry["b"] != ""):
    entries.append(entry)

In [100]:
len(entries) # 总条目数量

79893

In [30]:
# DEPRECATED len(blankinfo) # 条目名有问题的词条数量

75001

In [101]:
entries[1:50]

[('    ཚིག་མཛོད་ནི་དུས་རབས་རིམ་བྱུང་གི་ལོ་རྒྱུས་ཐོག་ཏུ་བེད་སྤྱོད་བྱས་དང་བྱེད་བཞིན་པའི་ཐ་སྙད་ཀྱི་བང་མཛོད་ཅིག་རེད།',
  '  འཇིག་རྟེན་འདིར་སྐད་ཡིག་ཡོད་པའི་མི་རིགས་ཤིག་ཡིན་ཕྱིན་རང་གི་སྐད་ཡིག་ཏུ་ཡོད་པའི་ཐ་སྙད་རྣམས་ཕྱོགས་སྒྲིག་དང་འགྲེལ་བཤད་བྱས་ནས་ཕྱི་རབས་རྗེས་འཛིན་པ་རྣམས་ཀྱི་ནོར་སྐལ་དུ་འཇོག་ཅིང་ཡོད་པ་ནི་འགྲོ་བ་མིའི་རིགས་ཐུན་མོང་གི་བྱ་གཞག་ཅིག་རེད།   བོད་ཀྱི་སྐད་ཡིག་ནི་ཆེས་གནའ་བོའི་དུས་ནས་བྱུང་པའི་དབྱངས་བཞི་དང༌།  གསལ་བྱེད་སུམ་ཅུ་འདུས་པའི་སྦྱོར་ཀློག་ཅན་གྱི་ཡི་གེ་ཅིག་ཡིན་སྟབས།  མིང་དང༌། ཚིག་གི་ཐ་སྙད་རྣམས་ཧ་ཅང་ཕུན་སུམ་ཚོགས་པོར་ཡོད་པ་སྟེ།  སྤྱི་ལོའི་སྔོན་གྱི་དུས་རབས་ཉི་ཤུ་པ་ཡས་མས་ཙམ་ནས་དམངས་ཁྲོད་ཀྱི་ཞིང་འབྲོག་གི་འཚོ་བའི་ཐད་ཀྱི་མིང་ཚིག་སྣ་འཛོམས་ཡོད་པ་མ་ཟད།  བོན་གྱི་མཁས་པ་རྣམས་ཀྱིས་མཛད་པའི་སྨན་དང༌། རྩིས་སོགས་རིག་གནས་ཀྱི་ཐ་སྙད་མང་པོ་བྱུང་བ་དང༌།  དེ་ནས་དུས་རབས་རིམ་བྱུང་གི་ནང་དུ་གཏོ་དང༌།  མདོས་དང༌།  ཡས་དང༌།  གླུད་བཅས་ཀྱི་ཆོ་ག་སྣ་ཚོགས་དར་སྤེལ་གཏོང་བ་དེས་ཆོས་དང་རིག་གནས་སྐོར་ཀྱི་ཐ་སྙད་ཇེ་མང་དུ་ཕྱིན་ཡོད།   སྤྱི་ལོའི་དུས་རབས་བདུན་པའི་ནང་ནས་བཟུང་རྒྱ་གར་གྱི་ཆོས་དང་རིག་གཞུང་རྣམས་བོད་ཀྱི་སྐད་ཡིག་ཏུ་བསྒྱུར་བ་ནས་བ

In [ ]:
############## This is a test: https://forum.freemdict.com/t/topic/4793/13?u=内存溢出的猫
entries = []
if True:
  soup = BeautifulSoup(xmls[5])
  ps = soup.select('p.Body')
  for p in ps:
    entry = {}
    span = p.span
    if span is not None:
      n = span.text
      if len(n) > 1:
        if n[-1] == "།":
          n = n[0:-1]
        if n[-1] == "་":
          n = n[0:-1]
    entry["a"] = n
    h = p.decode_contents()
    entry["b"] = h.split("</span>")[-1].strip()
    if entry["a"] != "" and entry["b"] != "":
      entries.append(entry)

In [ ]:
print(entries) # 不截最后一位的测试，仅第五页

[{'a': 'ཀ་ནི་སྒོ་བཞིའི་མཆོད་རྟེན།', 'b': 'ལྷ་སའི་གཙུག་ལག་ཁང་གི་བྱང་རྒྱུད་ཁྲོམ་ལམ་དུ་སྔར་ཡོད་མཆོད་རྟེན་སྒོ་བཞི་མའི་མིང་།'}, {'a': 'ཀ་གནམ།', 'b': 'སྤོ་བོ་ཡུལ་གྱི་ཀ་གནམ་ཞེས་པའི་ས་མིང་ཡིན།'}, {'a': 'ཀ་གནམ་རྒྱལ་པོ།', 'b': 'སྔར་སྤོ་བོ་ཡུལ་སྡེ་ལ་མངའ་དབང་བསྒྱུར་བའི་དཔོན་པོ་ཡིན།  འདིའི་རྒྱུད་པ་ནི་རྒྱལ་པོ་གྲི་གུམ་བཙན་པོའི་སྲས་གསུམ་ཀོང་པོའི་ཡུལ་དུ་བྲོས།  སྲས་ཤ་ཁྲི་སྤུ་མིའི་རྒྱལ་པོ་མཛད།  དེ་ནས་བཟུང་དེའི་བརྒྱུད་པ་རྣམས་ཀྱིས་ཡུལ་འདི་དབང་བསྒྱུུར་མཛད་པ་ལ་སྤོ་བོ་ཀ་གནམ་རྒྱལ་པོ་ཞེས་ཡོངས་སུ་གྲགས་པ་འདི་བྱུང་།  སྤྱི་ལོའི་༡༩༣༠ཡས་མས་ཙམ་བར་ཧ་ལམ་སྦོ་བོ་རང་སྡེ་རང་དཔོན་གྱི་རྣམ་པ་ལྟ་བུ་ཞིག་ཆགས་ཡོད་པ་རེད།  བོད་སྡེ་པ་གཞུང་ལ་སྐབས་འགར་ལོ་ཁྲལ་མཇལ་བ་དང་སྐབས་འགར་ལོ་ཁྲལ་མི་མཇལ་བ་ཁྲལ་རིགས་ལས་གཡོ་གང་ཐུབ་བྱེད་ཅིང་ཡོད་པ་རེད།  སྤྱི་ལོ་༡༩༣༠ལོ་ཙམ་ནས་སྡེ་པ་གཞུང་གིས་སྤོ་བོ་ཀ་གནམ་སྡེ་པ་ལ་དམག་དྲངས་ཏེ་དམག་འཁྲུག་ཉིན་ཁ་ཤས་བྱས་རྗེས།  ཀ་གནམ་རྒྱལ་པོ་དབང་ཆེན་བདུད་འདུལ་སྐུ་འཁོར་དང་བཅས་པ་རྒྱ་གར་ལ་བྲོས་བྱོལ་ཕྱིན་པ་དང་།  ཀ་གནམ་རྒྱལ་པོའི་ས་གཞིས་ཡུལ་གྲུ་རྣམས་སྡེ་པ་གཞུང་གིས་བདག་ཏུ་བཟུང་ཞིང་།  སྤོ་བོ་སྤྱི་ཁྱབ་ལས་ཁུང་བཙུགས་ཤིང་།  ཇ་དང་དམག་མིས་ས་སྲུང

In [ ]:
import json
dum = json.dumps(entries)
print(dum[0:1000])
with open("./drive/MyDrive/mdict/tibet/gj.txt",'w',encoding='utf-8') as fileobject:
  fileobject.write(dum)

[{"a": "\u0f67\u0f71\u0f74\u0f7e\u0f0b\u0f58\u0f5b\u0f51\u0f0d", "b": "\u0f5a\u0f72\u0f42\u0f0b\u0f58\u0f5b\u0f7c\u0f51\u0f0b\u0f53\u0f72\u0f0b\u0f51\u0f74\u0f66\u0f0b\u0f62\u0f56\u0f66\u0f0b\u0f62\u0f72\u0f58\u0f0b\u0f56\u0fb1\u0f74\u0f44\u0f0b\u0f42\u0f72\u0f0b\u0f63\u0f7c\u0f0b\u0f62\u0f92\u0fb1\u0f74\u0f66\u0f0b\u0f50\u0f7c\u0f42\u0f0b\u0f4f\u0f74\u0f0b\u0f56\u0f7a\u0f51\u0f0b\u0f66\u0fa4\u0fb1\u0f7c\u0f51\u0f0b\u0f56\u0fb1\u0f66\u0f0b\u0f51\u0f44\u0f0b\u0f56\u0fb1\u0f7a\u0f51\u0f0b\u0f56\u0f5e\u0f72\u0f53\u0f0b\u0f54\u0f60\u0f72\u0f0b\u0f50\u0f0b\u0f66\u0f99\u0f51\u0f0b\u0f40\u0fb1\u0f72\u0f0b\u0f56\u0f44\u0f0b\u0f58\u0f5b\u0f7c\u0f51\u0f0b\u0f45\u0f72\u0f42\u0f0b\u0f62\u0f7a\u0f51\u0f0d  \u0f60\u0f47\u0f72\u0f42\u0f0b\u0f62\u0f9f\u0f7a\u0f53\u0f0b\u0f60\u0f51\u0f72\u0f62\u0f0b\u0f66\u0f90\u0f51\u0f0b\u0f61\u0f72\u0f42\u0f0b\u0f61\u0f7c\u0f51\u0f0b\u0f54\u0f60\u0f72\u0f0b\u0f58\u0f72\u0f0b\u0f62\u0f72\u0f42\u0f66\u0f0b\u0f64\u0f72\u0f42\u0f0b\u0f61\u0f72\u0f53\u0f0b\u0f55\u0fb1\u0

In [102]:
thedict = {}
for x in entries:
  if x[0] not in thedict:
    thedict[x[0]] = x[1]

In [104]:
len(thedict)

77786

In [106]:
entries[78:122]

[('ཀ་མདའ་དགོན།',
  '②  མིང་གཞན་ཀི་ཀུ་ཁུ་བྱུག་དགོན་ཟེར། འདེབས་མཁན་དང་འདེབས་ལོ་མི་གསལ།  གྲུབ་མཐའ་དགེ་ལུགས་པ་ཡིན།    ད་ལྟ་བོད་རང་སྐྱོང་ལྗོངས་ཆབ་མདོ་ས་ཁུལ་སྨར་ཁམས་རྫོང་ཁོངས་སུ་ཡོད།'),
 ('ཀ་མདོ་བཀྲ་ཤིས་ཆོས་རྫོང་།',
  'ཕྱག་བཏབ་མཁན་དང་བཞེངས་དུས་མི་གསལ།    གྲུབ་མཐའ་དགེ་ལུགས་ཡིན།  ད་ལྟ་མཚོ་ཤར་བ་ཡན་རྫོང་ཁོངས་སུ་ཡོད།'),
 ('ཀ་ནི་ཀ',
  '〔ལེགས〕སྔར་རྒྱ་གར་གྱི་རྒྱལ་པོ་ཞིག་སྟེ།  དུས་རབས་དང་པོའི་མཇུག་ཏུ་དྲག་ཤུགས་ཀྱིས་རྒྱལ་ཕྲན་གཞན་རྣམས་བཏུལ་ནས་ཇན་ཏོ་ལ་ཞེས་པའི་རྒྱལ་ཁབ་བཙུགས།  ཚེ་མཇུག་ཏུ་སངས་ལ་རྒྱས་སྐྱབས་སུ་བསྟེན་ནས་དགོན་སྡེ་དང་མཆོད་རྟེན་མང་པོ་བཞེངས།  སློབ་དཔོན་རྟ་དབྱངས་བཤེས་གཉེན་དུ་བསྟེན་ནས་བསྟན་པ་སྲུང་སྐྱོང་བྱས།  ཁ་ཆེའི་ཡུལ་དུ་དགྲ་བཅོམ་པ་མང་པོ་གདན་དྲངས་ཏེ་སྡེ་སྣོད་གསུམ་ལས་བཏུས་པའི་བྱེ་བྲག་ཆེན་པོ་ཞེས་པའི་བསྟན་བཅོས་བསྒྲིགས།'),
 ('ཀ་ནི་སྒོ་བཞིའི་མཆོད་རྟེན།',
  'ལྷ་སའི་གཙུག་ལག་ཁང་གི་བྱང་རྒྱུད་ཁྲོམ་ལམ་དུ་སྔར་ཡོད་མཆོད་རྟེན་སྒོ་བཞི་མའི་མིང་།'),
 ('ཀ་གནམ།', 'སྤོ་བོ་ཡུལ་གྱི་ཀ་གནམ་ཞེས་པའི་ས་མིང་ཡིན།'),
 ('ཀ་གནམ་རྒྱལ་པོ།',
  'སྔར་སྤོ་བོ་ཡུལ་སྡེ་ལ་མངའ་དབང་བསྒྱུར་བའི་དཔོན་པོ་ཡིན།  འདིའི་རྒྱུད་པ་ནི་རྒྱལ་པོ་གྲི་གུམ་བཙན

In [105]:
from __future__ import unicode_literals
from writemdict import MDictWriter
writer = MDictWriter(thedict, title="Gangjong Tsigzhod Chenmo beta", description='For <a href="https://forum.freemdict.com/t/topic/4793/1">manjushri.</a>')
outfile = open("./drive/MyDrive/mdict/tibet/gangjong_v2.mdx", "wb")
writer.write(outfile)
outfile.close()

TypeError: ignored